<a href="https://colab.research.google.com/github/Megs130805/machine-learning/blob/main/Model_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp -r "/content/drive/MyDrive/Colab Notebooks/plant_village_datatset/archive (7)/PlantVillage/" /content/
dataset_path = "/content/PlantVillage/"

In [ ]:
# ===============================
# 🌿 PlantVillage: Model Comparison (5 Models)
# ===============================

!pip install tensorflow scikit-learn --quiet

import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2, VGG16, ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Conv2D, MaxPooling2D, Flatten, Input
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import time

# ===============================
# 1️⃣ Load Dataset
# ===============================
dataset_path = "/content/PlantVillage/"

datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_gen = datagen.flow_from_directory(
    dataset_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

val_gen = datagen.flow_from_directory(
    dataset_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

num_classes = train_gen.num_classes

# ===============================
# 2️⃣ Helper Function for Timing
# ===============================
def timed_train(model, train_gen, val_gen, epochs=5):
    start = time.time()
    history = model.fit(train_gen, validation_data=val_gen, epochs=epochs, verbose=1)
    duration = time.time() - start
    val_acc = history.history['val_accuracy'][-1]
    return val_acc, duration

results = {}

# ===============================
# 3️⃣ Model 1 – MobileNetV2 (Transfer Learning)
# ===============================
base_mobilenet = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224,224,3))
base_mobilenet.trainable = False

x = base_mobilenet.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.3)(x)
pred = Dense(num_classes, activation='softmax')(x)
model_mobilenet = Model(inputs=base_mobilenet.input, outputs=pred)

model_mobilenet.compile(optimizer=Adam(0.001), loss='categorical_crossentropy', metrics=['accuracy'])

acc_mob, time_mob = timed_train(model_mobilenet, train_gen, val_gen)
results["MobileNetV2"] = (acc_mob, time_mob)

# ===============================
# 4️⃣ Model 2 – Custom CNN (Simple)
# ===============================
model_cnn = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(224,224,3)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(num_classes, activation='softmax')
])

model_cnn.compile(optimizer=Adam(0.001), loss='categorical_crossentropy', metrics=['accuracy'])
acc_cnn, time_cnn = timed_train(model_cnn, train_gen, val_gen)
results["Custom CNN"] = (acc_cnn, time_cnn)

# ===============================
# 5️⃣ Model 3 – VGG16 (Frozen)
# ===============================
base_vgg = VGG16(weights='imagenet', include_top=False, input_shape=(224,224,3))
base_vgg.trainable = False

x = base_vgg.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.3)(x)
pred = Dense(num_classes, activation='softmax')(x)
model_vgg = Model(inputs=base_vgg.input, outputs=pred)

model_vgg.compile(optimizer=Adam(0.001), loss='categorical_crossentropy', metrics=['accuracy'])
acc_vgg, time_vgg = timed_train(model_vgg, train_gen, val_gen)
results["VGG16"] = (acc_vgg, time_vgg)

# ===============================
# 6️⃣ Model 4 – ResNet50 (Frozen)
# ===============================
base_resnet = ResNet50(weights='imagenet', include_top=False, input_shape=(224,224,3))
base_resnet.trainable = False

x = base_resnet.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.3)(x)
pred = Dense(num_classes, activation='softmax')(x)
model_resnet = Model(inputs=base_resnet.input, outputs=pred)

model_resnet.compile(optimizer=Adam(0.001), loss='categorical_crossentropy', metrics=['accuracy'])
acc_resnet, time_resnet = timed_train(model_resnet, train_gen, val_gen)
results["ResNet50"] = (acc_resnet, time_resnet)

# ===============================
# 7️⃣ Model 5 – SVM (on Flattened Images)
# ===============================
print("\nExtracting small subset for SVM (this might take a while)...")

# Smaller subset for CPU
X, y = [], []
for i in range(500):  # only first 500 images for speed
    img, label = train_gen.next()
    X.append(img[0].flatten())
    y.append(np.argmax(label[0]))
X, y = np.array(X), np.array(y)

svm = SVC(kernel='rbf')
start = time.time()
svm.fit(X, y)
duration_svm = time.time() - start
acc_svm = svm.score(X, y)
results["SVM (raw pixels)"] = (acc_svm, duration_svm)

# ===============================
# 8️⃣ Comparison Table
# ===============================
import pandas as pd
df = pd.DataFrame(results, index=["Validation Accuracy", "Training Time (sec)"]).T
df["Validation Accuracy"] = (df["Validation Accuracy"] * 100).round(2)
df["Training Time (min)"] = (df["Training Time (sec)"] / 60).round(2)
df.drop(columns=["Training Time (sec)"], inplace=True)
print("\n================= MODEL PERFORMANCE COMPARISON =================\n")
print(df)


Found 16516 images belonging to 15 classes.
Found 4122 images belonging to 15 classes.
Epoch 1/5
517/517 ━━━━━━━━━━━━━━━━━━━━ 78s 115ms/step - accuracy: 0.6560 - loss: 1.1216 - val_accuracy: 0.8806 - val_loss: 0.3598
Epoch 2/5
517/517 ━━━━━━━━━━━━━━━━━━━━ 36s 70ms/step - accuracy: 0.8655 - loss: 0.4149 - val_accuracy: 0.8950 - val_loss: 0.3052
Epoch 3/5
517/517 ━━━━━━━━━━━━━━━━━━━━ 32s 61ms/step - accuracy: 0.8971 - loss: 0.3161 - val_accuracy: 0.9064 - val_loss: 0.2779
Epoch 4/5
517/517 ━━━━━━━━━━━━━━━━━━━━ 31s 61ms/step - accuracy: 0.9166 - loss: 0.2519 - val_accuracy: 0.9192 - val_loss: 0.2452
Epoch 5/5
517/517 ━━━━━━━━━━━━━━━━━━━━ 32s 62ms/step - accuracy: 0.9220 - loss: 0.2279 - val_accuracy: 0.9199 - val_loss: 0.2341
Epoch 1/5


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


517/517 ━━━━━━━━━━━━━━━━━━━━ 49s 81ms/step - accuracy: 0.4218 - loss: 1.8944 - val_accuracy: 0.7785 - val_loss: 0.6762
Epoch 2/5
517/517 ━━━━━━━━━━━━━━━━━━━━ 36s 69ms/step - accuracy: 0.7543 - loss: 0.7371 - val_accuracy: 0.8508 - val_loss: 0.4448
Epoch 3/5
517/517 ━━━━━━━━━━━━━━━━━━━━ 40s 77ms/step - accuracy: 0.8412 - loss: 0.4786 - val_accuracy: 0.8797 - val_loss: 0.3678
Epoch 4/5
517/517 ━━━━━━━━━━━━━━━━━━━━ 36s 69ms/step - accuracy: 0.8822 - loss: 0.3503 - val_accuracy: 0.8845 - val_loss: 0.3602
Epoch 5/5
517/517 ━━━━━━━━━━━━━━━━━━━━ 38s 73ms/step - accuracy: 0.9149 - loss: 0.2565 - val_accuracy: 0.8806 - val_loss: 0.3718
58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/5
517/517 ━━━━━━━━━━━━━━━━━━━━ 134s 232ms/step - accuracy: 0.3655 - loss: 2.0494 - val_accuracy: 0.7213 - val_loss: 1.0049
Epoch 2/5
517/517 ━━━━━━━━━━━━━━━━━━━━ 105s 203ms/step - accuracy: 0.6791 - loss: 1.0366 - val_accuracy: 0.7863 - val_loss: 0.7182
Epoch 3/5
517/517 ━━━━━━━━━━━━━━━━━━━━ 105s 203ms/st

AttributeError: 'DirectoryIterator' object has no attribute 'next'

In [ ]:
# ===============================
# 🧠 Run SVM Separately & Add to Results
# ===============================

from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import numpy as np
import time

print("\nRunning SVM separately (subset of training images)...")

X, y = [], []
subset_size = 500  # can increase if system can handle more
train_gen.reset()  # reset generator to start from beginning

for i in range(subset_size):
    img, label = next(train_gen)
    X.append(img[0].flatten())
    y.append(np.argmax(label[0]))

X, y = np.array(X), np.array(y)

print("Training SVM...")
svm = SVC(kernel='rbf')
start = time.time()
svm.fit(X, y)
duration_svm = time.time() - start
acc_svm = svm.score(X, y)

# Save results without losing previous ones
results["SVM (raw pixels)"] = (acc_svm, duration_svm)

# ===============================
# 🔍 Show Updated Comparison Table
# ===============================
import pandas as pd

df = pd.DataFrame(results, index=["Validation Accuracy", "Training Time (sec)"]).T
df["Validation Accuracy"] = (df["Validation Accuracy"] * 100).round(2)
df["Training Time (min)"] = (df["Training Time (sec)"] / 60).round(2)
df.drop(columns=["Training Time (sec)"], inplace=True)

print("\n================= UPDATED MODEL PERFORMANCE COMPARISON =================\n")
print(df)



Running SVM separately (subset of training images)...
Training SVM...

================= UPDATED MODEL PERFORMANCE COMPARISON =================

                  Validation Accuracy  Training Time (min)
MobileNetV2                     91.99                 3.49
Custom CNN                      88.06                 3.30
VGG16                           84.50                 9.23
ResNet50                        40.68                 4.91
SVM (raw pixels)                91.80                 1.27


In [ ]:
import tensorflow as tf
print("Available devices:")
print(tf.config.list_physical_devices())


Available devices:
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
